In [1]:
from ioh import get_problem, ProblemClass
from ioh import logger
import math
import os
from dataclasses import dataclass
import BA 
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.random_projection import GaussianRandomProjection
import botorch
import gpytorch
from gpytorch.constraints import Interval
from gpytorch.kernels import MaternKernel, ScaleKernel, RBFKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from torch.quasirandom import SobolEngine

from botorch.acquisition.analytic import LogExpectedImprovement
from botorch.exceptions import ModelFittingError
from botorch.fit import fit_gpytorch_mll
from botorch.generation import MaxPosteriorSampling
from botorch.models import SingleTaskGP
from botorch.optim import optimize_acqf
from sklearn.preprocessing import StandardScaler


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {device}")
dtype = torch.float
SMOKE_TEST = os.environ.get("SMOKE_TEST")

# problem dimension
dim = 40 if not SMOKE_TEST else 50
projected_dim = 15

# initial sample points
n_init = 10 if not SMOKE_TEST else 4
max_cholesky_size = float("inf")

problem = get_problem(1, dimension=dim, instance=2, problem_class=ProblemClass.BBOB)

Running on cpu


In [3]:
EVALUATION_BUDGET = 100 if not SMOKE_TEST else 10
NUM_RESTARTS = 3 if not SMOKE_TEST else 2
RAW_SAMPLES = 51 if not SMOKE_TEST else 4
N_CANDIDATES = min(dim, max(200, 20 * dim)) if not SMOKE_TEST else 4

In [4]:
n_sample = 10
X_sample = BA.get_initial_points(40, n_sample)
S = BA.embedding_matrix_kpca(input_dim=40, target_dim=1, data=X_sample)
X = torch.randint(100, (7, 2))
print(f"S before increase\n{S}")
print(f"X before increase\n{X}")

S before increase
tensor([[-0.2470,  0.2353,  0.0755,  0.0502,  0.1025, -0.2266, -0.1951, -0.0364,
          0.0376, -0.1764, -0.1356,  0.1051,  0.0400, -0.2275, -0.2758,  0.0786,
         -0.1723, -0.1697, -0.0017,  0.0832, -0.0964,  0.2643,  0.2169,  0.1801,
         -0.0524, -0.0026,  0.3409,  0.1895,  0.1569,  0.0359,  0.2454, -0.2445,
          0.0111, -0.0142,  0.1801,  0.0800,  0.0426, -0.0258, -0.0829, -0.0572]])
X before increase
tensor([[98, 68],
        [27, 67],
        [23, 81],
        [46, 32],
        [97, 88],
        [84, 23],
        [84, 65]])


In [5]:
state = BA.BaxusState(dim=dim, eval_budget=EVALUATION_BUDGET - n_init)
n_sample = 40
X_sample = BA.get_initial_points(state.d_init, n_sample)
S = BA.embedding_matrix_kpca(input_dim=state.dim, target_dim=state.d_init, data=X_sample)

X_baxus_target = BA.get_initial_points(state.d_init, n_init)
X_baxus_input = X_baxus_target @ S

Y_baxus = torch.tensor(
        [-problem(list(x)) for x in X_baxus_input], dtype=dtype, device=device
    ).unsqueeze(-1)
print(Y_baxus.shape)
with botorch.settings.validate_input_scaling(False):
    for _ in range(EVALUATION_BUDGET - n_init):  # Run until evaluation budget depleted
            # Fit a GP model
            train_Y = (Y_baxus - Y_baxus.mean()) / Y_baxus.std()
            likelihood = GaussianLikelihood(noise_constraint=Interval(1e-8, 1e-3))
            model = SingleTaskGP(
                X_baxus_target, train_Y, likelihood=likelihood
            )
            mll = ExactMarginalLogLikelihood(model.likelihood, model)

            # Fit the model using Cholesky context
            with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                try:
                    fit_gpytorch_mll(mll)
                except ModelFittingError:
                    # Use Adam-based optimization if Cholesky decomposition fails
                    optimizer = torch.optim.Adam([{"params": model.parameters()}], lr=0.1)
                    for _ in range(200):
                        optimizer.zero_grad()
                        output = model(X_baxus_target)
                        loss = -mll(output, train_Y.flatten())
                        loss.backward()
                        optimizer.step()

                # Generate new candidates
                X_next_target = BA.create_candidate(
                    state=state,
                    model=model,
                    X=X_baxus_target,
                    Y=train_Y,
                    device=device,
                    dtype=dtype,
                    n_candidates=N_CANDIDATES,
                    num_restarts=NUM_RESTARTS,
                    raw_samples=RAW_SAMPLES,
                    acqf="ts",
                )
            
            # Map new candidates to high-dimensional space
            X_next_input = X_next_target @ S
            Y_next = torch.tensor(
                [-problem(list(x)) for x in X_next_input], dtype=dtype, device=device
            ).unsqueeze(-1)

            # Update state and concatenate new points
            state = BA.update_state(state=state, Y_next=Y_next)
            X_baxus_input = torch.cat((X_baxus_input, X_next_input), dim=0)
            X_baxus_target = torch.cat((X_baxus_target, X_next_target), dim=0)
            Y_baxus = torch.cat((Y_baxus, Y_next), dim=0)
            
            print(
                f"iteration {len(X_baxus_input)}, d={len(X_baxus_target.T)})  Best value: {state.best_value:.3}, TR length: {state.length:.3}"
            )

            if state.restart_triggered:
                state.restart_triggered = False
                print("increasing target space")

                S, X_baxus_target = BA.increase_embedding_and_observations(
                    S, X_baxus_target, state.new_bins_on_split
                )
                print(f"new dimensionality: {len(S)}")
                
                state.target_dim = len(S)
                state.length = state.length_init
                state.failure_counter = 0
                state.success_counter = 0

torch.Size([10, 1])


/var/folders/yk/hp80cpj103xfzhptqczygcv40000gn/T/ipykernel_8901/4094046806.py:18: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/var/folders/yk/hp80cpj103xfzhptqczygcv40000gn/T/ipykernel_8

iteration 11, d=1)  Best value: -6.77e+02, TR length: 0.4
iteration 12, d=1)  Best value: -6.77e+02, TR length: 0.2
iteration 13, d=1)  Best value: -6.77e+02, TR length: 0.1


/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/var/folders/yk/hp80cpj103xfzhptqczygcv40000gn/T/ipykernel_8901/4094046806.py:18: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision a

iteration 14, d=1)  Best value: -6.77e+02, TR length: 0.05
iteration 15, d=1)  Best value: -6.77e+02, TR length: 0.025
iteration 16, d=1)  Best value: -6.77e+02, TR length: 0.0125


...

/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/var/folders/yk/hp80cpj103xfzhptqczygcv40000gn/T/ipykernel_8901/4094046806.py:18: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-pack

iteration 48, d=27)  Best value: -6.19e+02, TR length: 0.8
iteration 49, d=27)  Best value: -6.19e+02, TR length: 0.8


/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/var/folders/yk/hp80cpj103xfzhptqczygcv40000gn/T/ipykernel_8901/4094046806.py:18: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(


iteration 50, d=27)  Best value: -6.19e+02, TR length: 0.8


KeyboardInterrupt: 